In [1]:
import pandas as pd
import os
import awswrangler as wr
env = os.environ.get('Environment')
print(f"Environment: {env!r}")


Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

In [3]:
%%athena_to_df --database f"{env}-luna-pharma-prepared-data" --out df_test 
select * from (select ar.group_id, ar.categorie, ar.code_article, ar.libelle,
    jh.job_name, jh.sw_no_lot, jh.browse_description, jh.sw_job_type, jh.job_status, 
    sa.id_numeric, sa.status as sample_status, sa.sample_type, sa.recd_date,
    te.analysis, te.test_number, sa.id_text, te.status
from test te            
join sample sa on sa.id_numeric = te.sample
join job_header jh on jh.job_name = sa.job_name
join sw_article ar on ar.code_article = jh.sw_article
where ar.group_id like 'ARA%'
and jh.sw_lot_validation = 'F'
and jh.sw_no_lot not like '%FOR%'
and sa.status <> 'U'
and from_iso8601_timestamp(sa.recd_date) BETWEEN current_date - INTERVAL '1' day AND current_date
and te.analysis <> 'DBL_ENTRY');

In [4]:
df_test.head(5)

,group_id,categorie,code_article,libelle,job_name,sw_no_lot,browse_description,sw_job_type,job_status,id_numeric,sample_status,sample_type,recd_date,analysis,test_number,id_text,status
0,ARAMON,PF,ARA_362671,rhC1 INH Solution API,ARA_362671_2023_01,DAR1035,Job created on 11-SEP-2023 15:06 by SAUVAGE us...,SOG,V,3007041,V,SOG,2024-06-12T15:14:02,ARA_BMREPAR1_2,16353054,S-ARA_362671_2023_01-475,P
1,ARAMON,PF,ARA_362671,rhC1 INH Solution API,ARA_362671_2023_01,DAR1035,Job created on 11-SEP-2023 15:06 by SAUVAGE us...,SOG,V,3007041,V,SOG,2024-06-12T15:14:02,ARA_VERDATE2_1,16353055,S-ARA_362671_2023_01-475,V
2,ARAMON,PF,ARA_362671,rhC1 INH Solution API,ARA_362671_2023_01,DAR1035,Job created on 11-SEP-2023 15:06 by SAUVAGE us...,SOG,V,3007041,V,SOG,2024-06-12T15:14:02,ARA_ACASPUS3_2,16353056,S-ARA_362671_2023_01-475,V
3,ARAMON,PF,ARA_362671,rhC1 INH Solution API,ARA_362671_2023_01,DAR1035,Job created on 11-SEP-2023 15:06 by SAUVAGE us...,SOG,V,3007041,V,SOG,2024-06-12T15:14:02,ARA_REPART6_6,16353057,S-ARA_362671_2023_01-475,P
4,ARAMON,PF,ARA_362671,rhC1 INH Solution API,ARA_362671_2023_01,DAR1035,Job created on 11-SEP-2023 15:06 by SAUVAGE us...,SOG,V,3007041,V,SOG,2024-06-12T15:14:02,ARA_REPART7_7,16353058,S-ARA_362671_2023_01-475,P


In [5]:
database_name = f'prod-dmdg-workspace-public-aramon-smartqc-20221012135705242'

In [6]:
table_name = f'{env}_ARA_luna_smartqc'

In [7]:
path = f's3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-20221012135705242'

In [8]:
workspace_prefix =f'public-aramon-smartqc-20221012135705242'

In [9]:
wr.s3.to_parquet(
    df=df_test,
    path=path,
    database=database_name,
    dataset=True,
    table=table_name,
    mode='overwrite',
    index=False,
    compression=None,
)

{'paths': ['s3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-20221012135705242/e66a2de2d0f04647b35db56edaa5a102.parquet'],
 'partitions_values': {}}

In [10]:
df_test.shape

(80, 17)